Welcome to the SNCAST jupyter notebook!

First lets load some libraries and default settings.
Please don't change anything in the cell below.

Click into the next cell and select 'Run' in the menu above, this will execute the python code in the cell.

In [ ]:
%matplotlib inline
import numpy as np
import sys
from obspy.signal.util import util_geo_km
from math import pow, log10, sqrt
%reload_ext autoreload
%autoreload 1
%aimport sncast

First we need an input file that contains four comma separated columns containing for each seismic station:

    longitude, latitude, noise [nm], station name
         
    for example: -7.5100, 55.0700, 0.53, IDGL
    
Let's use an example file for Ireland, called "noisedata". If you want to use your own data replace the filename in the next cell.

Clicking into the next cell and selecting 'Run' in the menu above will execute the SNCAST python code.

In [ ]:
sncast.minML('noisedata')

An output file will be generated with the ending ".grd". The exact filename depends on the parameters you have chosen. In the example above no parameters were chosen and hence default values were applied. Besides a longitude and latitude range applicable for Ireland the following default settings apply:

stat_num = 4	required number of station detections
snr = 3			required signal-to-noise ratio for detection
foc_depth = 0	assumed focal event depth in km
region = CAL	locality for assumed ML scale parameters ('UK' or 'CAL')

With these default settings the output file will be called "noisedata-stat4-foc0-snr3-CAL.grd"

Running the following cell will plot a contour map of the results, it should look like this: ![title](https://homepages.dias.ie/martin/sncast-example1.png)

In [ ]:
from scipy.interpolate import griddata
import numpy as np
import matplotlib.pyplot as plt

from scipy.interpolate import griddata
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

x, y, z = np.loadtxt('noisedata-stat4-foc0-snr3-CAL.grd').T

# define grid
xi = np.linspace(-11.5,-4.5,15)
yi = np.linspace(51,56,15)

X,Y= np.meshgrid(xi,yi)

# grid the data
Z = griddata((x,y),z,(X,Y),method='linear')

# contour the gridded data, 
plt.contour(X,Y,Z,linewidths=0.5,colors='k')
plt.contourf(X,Y,Z,cmap=plt.cm.hot_r, vmax=(Z).max(), vmin=(Z).min())

plt.colorbar() # draw colorbar
plt.title('Minimum detectable ML')
plt.xlabel('Longitude [degrees]')
plt.ylabel('Latitude [degrees]')

# mark station locations on map
xs, ys, zs, stat = np.genfromtxt('noisedata',delimiter=',').T
plt.autoscale(False)
plt.plot(xs,ys, marker='v',color='k',linestyle='none');


Now let's see how to run SNCAST with non-default settings. Let's for example use different latitude and longitude settings to get a larger map extent and use stricter detection limits stat_num=6 and snr=4. You can see the definitions and order of parameters that sncast.minML accepts in the main code sncast.py.


In [ ]:
sncast.minML('noisedata','./',-12.5,-4.5,48.5,58.5,0.4,0.25,6,4)

This creates an output file called "noisedata-stat6-foc0-snr4-CAL.grd". Let's plot it by running code similar as above. The only changes are the input filename and the latitude longitude changes (xi, yi).

In [ ]:
from scipy.interpolate import griddata
import numpy as np
import matplotlib.pyplot as plt

from scipy.interpolate import griddata
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

x, y, z = np.loadtxt('noisedata-stat6-foc0-snr4-CAL.grd').T

# define grid
xi = np.linspace(-12.5,-4.5,15)
yi = np.linspace(48.5,58.5,15)

X,Y= np.meshgrid(xi,yi)

# grid the data
Z = griddata((x,y),z,(X,Y),method='linear')

# contour the gridded data, 
plt.contour(X,Y,Z,linewidths=0.5,colors='k')
plt.contourf(X,Y,Z,cmap=plt.cm.hot_r, vmax=(Z).max(), vmin=(Z).min())

plt.colorbar() # draw colorbar
plt.title('Minimum detectable ML')
plt.xlabel('Longitude [degrees]')
plt.ylabel('Latitude [degrees]')

# mark station locations on map
xs, ys, zs, stat = np.genfromtxt('noisedata',delimiter=',').T
plt.autoscale(False)
plt.plot(xs,ys, marker='v',color='k',linestyle='none');


With the stricter detection threshold settings of stat_num=6 and snr=4 the resulting map shows that the ML detection thresold changed significantly, note the different colour scale in the two figures!

This demonstrates that SNCAST is not the best tool to determine the absolute value of a network's detection threshold. We recommend to independently calculate the magnitude of completeness (Mc) of the network and use the result to determine suitable values for stat_num and snr. Basically tune SNCAST by changing the two values (and maybe what type of station noise levels you use) until the average detection limit in the map corresponds with Mc. Then SNCAST can be used as a powerful tool to determine geopgraphical and also temporal changes in network capability. For more information see the SNCAST publication at  http://dx.doi.org/10.1007/s10950-019-09819-0 or in https://www.researchgate.net/profile/Martin_Moellhoff2